## Image Processing with Dense Layers

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

## CIFAR-10 View Images

In [0]:
%matplotlib inline
from keras.datasets import cifar10
import matplotlib.pyplot as plt

# List of names for each CIFAR10 class
cifar10_class_names = {
    0: "Plane",
    1: "Car",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Boat",
    9: "Truck"
}

# Load the entire data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Loop through each picture in the data set
for i in range(10):
    # Grab an image from the data set
    sample_image = x_train[i]
    # Grab the image's expected class id
    image_class_number = y_train[i][0]
    # Look up the class name from the class id
    image_class_name = cifar10_class_names[image_class_number]

    # Draw the image as a plot
    plt.imshow(sample_image)
    # Label the image
    plt.title(image_class_name)
    # Show the plot on the screen
    plt.show()

## Classifying CIFAR-10 Data

In [0]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
x_train.shape,x_test.shape

In [0]:
y_test

In [0]:
x_train = x_train.reshape(50000,3072)
x_test = x_test.reshape(10000,3072)

In [0]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [0]:
y_test

In [0]:
# Create a model and add layers
model = Sequential()

model.add(Dense(3000, activation="sigmoid",input_shape=(3072,)))
model.add(Dense(2000, activation="relu"))
model.add(Dense(500, activation="relu"))
model.add(Dense(10, activation="softmax"))

print(model.summary())

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=10,
    validation_data=(x_test, y_test)
    )

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

In [0]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("/content/sample_data/dog.png", target_size=(32,32))
#img = image.load_img("car.png", target_size=(32,32))
#img = image.load_img("dog.png", target_size=(32,32))
#img = image.load_img("frog.png", target_size=(32,32))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

test_img = image_to_test.reshape((1,3072))
img_class = model.predict_classes(test_img)
prediction = img_class[0]
prediction,class_labels[prediction]